# HW1: COVID-19 Cases Prediction (Regression)

In [62]:
!nvidia-smi

Tue Oct 24 22:36:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.86       Driver Version: 526.86       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   49C    P8    24W / 370W |   2006MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download data

In [63]:
# google drive link
# !pip install gdown
# !gdown --id '1BjXalPZxq9mybPKNjF3h5L3NcF7XKTS-' --output covid_train.csv
# !gdown --id '1B55t74Jg2E5FCsKCsUEkPKIuqaY7UIi1' --output covid_test.csv

# dropbox link (google-colab)
# !wget -O covid_train.csv https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0
# !wget -O covid_test.csv https://www.dropbox.com/s/zalbw42lu4nmhr2/covid.test.csv?dl=0

# dropbox link (jupyter)
# !python -m wget https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0 -o covid_train.csv
# !python -m wget https://www.dropbox.com/s/zalbw42lu4nmhr2/covid.test.csv?dl=0 -o covid_test.csv


## Import packages

In [64]:
# Numerical Operations
import  math
import numpy as np

#Reading/Writing Data
import pandas as pd
import os
import csv

#For Progress Bar
from tqdm import tqdm

#Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

#For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

## Some Utility Functions
You do not need to modify this part.

In [65]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True    #cudnn(CUDA Deep Neural Network) 為加速神經網路的library,設置deterministic=true,隨機性將被禁用
    torch.backends.cudnn.benchmark = False       #關閉cudnn的性能優化，cudnn會通過優化convolution等操作來提升性能，但可能導致運行結果不一致。
    np.random.seed(seed)                         #設定NumPy函式庫中的隨機數產生器的種子，以確保NumPy產生的隨機數與隨機性運算的順序無關。 
    torch.manual_seed(seed)                      #設定PyTorch的隨機數產生器的種子，以確保模型的初始化、權重初始化等操作是確定性的。
    if torch.cuda.is_available():                #檢查可用的GPU
        torch.cuda.manual_seed_all(seed)         #設定所有GPU裝置上的隨機數產生器的種子，以確保在使用GPU進行訓練時的結果可重複。
            
def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

#
def predict(test_loader, model, device):
    '''Make predictions on test data set, and no resources are wasted to compute gradients  '''
    model.eval() # Set your model to evaluation mode. Disables Dropout and batch normalization值不變, 也不計算gradient
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   #detach():預測階段不保留gradient訊息(減少內存消耗), cpu(): 將data從GPU移動到CPU(方便後續numpy操作)
    preds = torch.cat(preds, dim=0).numpy()     #convert to numpy array
    return preds
                            

## Dataset

In [66]:
class COVID19Dataset(Dataset):          #Inheritance Pytorch Dataset
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):      #constructor
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

## Neural Network Model
Try out different model architectures by modifying the class below.

In [67]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()   #call parent class(nn.Module) __init__ constructor
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),  #fully-connected layer
            nn.ReLU(),                 #activation function
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )
        
    def forward(self, x):
        '''Forward pass: compute ouput of NN'''
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B) , remove the specified dimension with length = 1
        return x

## Feature Selection 
Choose features you deem useful by modifying the function below.

In [68]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = list(range(35, raw_x_train.shape[1])) # TODO: Select suitable feature columns.
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

## Training Loop

In [6]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.7) 
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    #Neural Network Training Loop
    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Reset gradients of model parameters.
            x, y = x.to(device), y.to(device)   # Move your data to device(GPU or CPU). 
            pred = model(x)                     # Input x to model to get pred
            loss = criterion(pred, y)           # pred & y(期望值) 計算loss
            loss.backward()                     # Compute gradient(backpropagation) of predictions loss. (計算loss的gradient)
            optimizer.step()                    # Update model parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)  #compute averaged loss
        writer.add_scalar('Loss/train', mean_train_loss, step) #visualize 
        
        #Neural Network Validation Loop
        model.eval() # Set your model to evaluation mode.
        loss_record = []  
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():             #在做evaluation/testing時, 不希望model學習到資料的內容以及update參數,所以關掉計算gradient
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

## Plot learning curves with 'tensorboard'
visualize your training progress

In [4]:
%reload_ext tensorboard
%tensorboard --logdir='./runs/'

Reusing TensorBoard on port 6006 (pid 42096), started 0:06:48 ago. (Use '!kill 42096' to kill it.)

## Configurations
config contains hyper-parameters for training and the path to save your model.

In [70]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 1212,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 3000,     # Number of epochs.            
    'batch_size': 256, 
    'learning_rate': 1e-5,              
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

## Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [71]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 3009 x 89 (35 states + 18 features x 3 days) 
# test_data size: 997 x 88 (without last day's positive rate)
train_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)  #disable shuffle when testing

train_data size: (2408, 89) 
valid_data size: (601, 89) 
test_data size: (997, 88)
number of features: 88


## Start training

In [72]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/3000]: 100%|██████████| 10/10 [00:00<00:00, 210.48it/s, loss=94.4]


Epoch [1/3000]: Train loss: 258.2040, Valid loss: 97.8839
Saving model with loss 97.884...


Epoch [2/3000]: 100%|██████████| 10/10 [00:00<00:00, 249.11it/s, loss=77.7]


Epoch [2/3000]: Train loss: 88.7831, Valid loss: 86.1965
Saving model with loss 86.196...


Epoch [3/3000]: 100%|██████████| 10/10 [00:00<00:00, 259.08it/s, loss=73.8]


...

Epoch [2998/3000]: Train loss: 1.6776, Valid loss: 1.4009


Epoch [2999/3000]: 100%|██████████| 10/10 [00:00<00:00, 278.48it/s, loss=1.44]


Epoch [2999/3000]: Train loss: 1.5323, Valid loss: 1.3590


Epoch [3000/3000]: 100%|██████████| 10/10 [00:00<00:00, 295.21it/s, loss=1.64]

Epoch [3000/3000]: Train loss: 1.4302, Valid loss: 1.7834


## Testing
The predictions of your model on testing set will be stored at pred.csv.

In [73]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')         

100%|██████████| 4/4 [00:00<00:00, 560.23it/s]


## Download
Run this block to download the pred.csv by clicking.

In [74]:
from IPython.display import FileLink
FileLink(r'pred.csv')

C:\Users\USER\Desktop\ML2023\homework\HW1_Regression\pred.csv